In [ ]:
import json

with open("results/tool-calling-Llama-3.3-70B-Instruct-0.0_range_0--1_user-azure_gpt-4o-llm_0710011037.json") as f:
    d =json.load(f)

In [8]:
dist = []
for task in d:
    traj = task['traj']
    for msg in traj:
        if msg['role'] == 'assistant' and msg['tool_calls'] :
            dist.append(len(msg['tool_calls']))

In [8]:
for a,b,c in zip([1,2,3],[4,5,6],[7,8,9]):
    print(a, b, c)

1 4 7
2 5 8
3 6 9


In [106]:
def extract_func(content):
    import re
    res = re.findall(r"\[{.*?}\]", content, re.DOTALL)
    return res[0] if len(res) > 0 else ''

s = "Great, Sofia. I have your item ID and the details you provided. Let's proceed with the exchange. First, I need to check if the coffee maker you received is indeed in a delivered status and if we have a suitable replacement available.\n\nI'll start by checking the status of the order associated with item ID #W7736983. Please wait a moment while I get this information for you.\n[{\"name\": \"get_order_details\", \"arguments\": {\"order_id\": \"#W7736983\"}][{\"name\": \"get_order_details\", \"arguments\": {\"order_id\": \"#W7736983\"}]\n"

type(extract_func(s))

str

In [ ]:
def process_wmdata_to_ftdata_by_task(obj):
    assert len(obj['expert_obs']) == len(obj['expert_acts']) == len(obj['explore_obs']) == len(obj['explore_acts'])
    conversations = []
    for expert_observation, expert_action, explore_observations, explore_actions in zip(obj['expert_obs'], obj['expert_acts'], obj['explore_obs'], obj['explore_acts']):
        wrapping_prompt = '''You are having a conversation with a user to help resolve a retail-relevant request. The previous message you receive is:\n{expert_observation}\n\nNow you have decided to respond with\n{response}.\n\n Please predict the next message that you will receive.'''
        if len(explore_observations) != len(explore_actions):
            continue
        expert_action_func = extract_func(expert_action['content'])
        for explore_observation, explore_action in zip(explore_observations, explore_actions):
            action = explore_action['content']
            action_func = extract_func(action)
            if action_func and expert_action_func:
                try:
                    if json.loads(expert_action_func) == json.loads(action_func):
                        continue
                except Exception as e:
                    # if there is any error on getting the func call structure, we skip it, even though it might be potention duplication
                    print(e)
                    print("1", expert_action_func)
                    print("2", action_func)
                    print("------")

            sft_user = {
                "from": "human",
                "value": wrapping_prompt.format(expert_observation=json.dumps(expert_observation, indent=4), response=action if not extract_func(action) else extract_func(action))
            }
            sft_response = {
                "from": "gpt",
                "value": json.dumps(explore_observation, indent=4)
            }
            conversations.append({"conversations": [sft_user, sft_response]})
    return conversations





In [108]:
import json
def read_jsonl(file_path):
    """Yield one JSON object per line from a JSONL file."""
    with open(file_path, 'r') as file:
        for line in file:
            yield json.loads(line)
# Example usage:
conversations = []
count = 0
for idx, obj in enumerate(read_jsonl('data/wm_datas.jsonl')):
    # try:
    count += len(obj['expert_obs'])
    conversations.extend(process_wmdata_to_ftdata_by_task(obj))
    # except Exception as e:
    #     print(e)
    #     print(obj['task_id'])
    #     continue

print(count)

Expecting ',' delimiter: line 1 column 157 (char 156)
1 [{"name": "create_new_order", "arguments": {"user_id": "harper_thomas_9402", "items": [{"product_id": "4635925001", "item_id": "5586947715", "quantity": 1}]
2 [{"name": "modify_pending_order_items", "arguments": {"order_id": "#W7425646", "item_ids": ["7510236436"], "new_item_ids": ["5586947715"], "payment_method_id": "credit_card_1283450"}}]
------
Expecting ',' delimiter: line 1 column 157 (char 156)
1 [{"name": "create_new_order", "arguments": {"user_id": "harper_thomas_9402", "items": [{"product_id": "4635925001", "item_id": "5586947715", "quantity": 1}]
2 [{"name": "get_product_details", "arguments": {"product_id": "4635925001"}}]
------
Expecting ',' delimiter: line 1 column 157 (char 156)
1 [{"name": "create_new_order", "arguments": {"user_id": "harper_thomas_9402", "items": [{"product_id": "4635925001", "item_id": "5586947715", "quantity": 1}]
2 [{"name": "modify_pending_order_items", "arguments": {"order_id": "#W7425646", 

In [109]:
len(conversations)

3074

In [68]:
print("Great, Sofia. I have your item ID and the details you provided. Let's proceed with the exchange. First, I need to check if the coffee maker you received is indeed in a delivered status and if we have a suitable replacement available.\n\nI'll start by checking the status of the order associated with item ID #W7736983. Please wait a moment while I get this information for you.\n[{\"name\": \"get_order_details\", \"arguments\": {\"order_id\": \"#W7736983\"}}]\n")

Great, Sofia. I have your item ID and the details you provided. Let's proceed with the exchange. First, I need to check if the coffee maker you received is indeed in a delivered status and if we have a suitable replacement available.

I'll start by checking the status of the order associated with item ID #W7736983. Please wait a moment while I get this information for you.
[{"name": "get_order_details", "arguments": {"order_id": "#W7736983"}}]



In [67]:
print(conversations[131]['conversations'][0]['value'])

You are having a conversation with a user to help resolve a retail-relevant request. The previous message you receive is:
{
    "role": "user",
    "content": "I have the item ID for the coffee maker I want to exchange. It's #W7736983. The coffee maker I received is black, has a 4-cup capacity, and is an espresso type with a timer feature. I would like to exchange it for a stainless steel drip coffee maker that has a built-in grinder."
}

Now you have decided to respond with
{
    "content": "Great, Sofia. I have your item ID and the details you provided. Let's proceed with the exchange. First, I need to check if the coffee maker you received is indeed in a delivered status and if we have a suitable replacement available.\n\nI'll start by checking the status of the order associated with item ID #W7736983. Please wait a moment while I get this information for you.\n[{\"name\": \"get_order_details\", \"arguments\": {\"order_id\": \"#W7736983\"}}]\n",
    "refusal": null,
    "role": "ass

In [ ]:
json.dump(conversations, open("data/wm_sft.json", 'w'), indent=4)